In [1]:
import random
random.seed(10)

In [2]:
import re
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from random import *

# **Loading Text**

In [3]:
from google.colab import drive

drive.mount('/content/drive')

import os

path = '/content/drive/MyDrive/Colab Notebooks/Cap11'

Mounted at /content/drive


In [4]:
text = open(os.path.join(path, 'texto.txt'), 'r').read()

# **Data PreProcessing**

In [5]:
sentences = re.sub('[,.!?\\-]', '', text.lower()).split('\n')

In [ ]:
print(sentences)

In [6]:
word_list = list(set(' '.join(sentences).split()))

In [ ]:
print(word_list)

In [7]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

In [ ]:
word_dict

In [8]:
for i, w in enumerate(word_list):
    word_dict[w] = i + 4

In [ ]:
print(word_dict)

In [9]:
number_dict = {i: w for i, w in enumerate(word_dict)}

In [ ]:
print(number_dict)

In [10]:
vocab_size = len(word_dict)
print(vocab_size)

70


In [11]:
token_list = list()

In [12]:
for sentence in sentences:
  arr = [word_dict[s] for s in sentence.split()]
  token_list.append(arr)

In [ ]:
text[0:29]

In [ ]:
token_list[0]

# **Hyperparameters**

In [13]:
batch_size = 6
n_segments = 2
dropout = 0.2

max_len = 100

max_pred = 7

n_layers = 6

n_heads = 12

d_model = 768

d_ff = d_model * 4

d_k = d_v = 64

NUM_EPOCHS = 50

In [15]:
def make_batch():
  batch = []

  positive = negative = 0

  while positive != batch_size / 2 or negative != batch_size / 2:
    tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))

    tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]

    input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]

    segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (1 + len(tokens_b))

    n_pred = min(max_pred, max(1, int(round(len(input_ids) * 0.15))))

    cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]

    shuffle(cand_maked_pos)

    masked_tokens, masked_pos = [], []

    for pos in cand_maked_pos[:n_pred]:
      masked_pos.append(pos)
      masked_tokens.append(input_ids[pos])

      if random() < 0.8:
        input_ids[pos] = word_dict['[MASK]']
      elif random() < 0.5:
        index = randint(0, vocab_size - 1)
        input_ids[pos] = word_dict[number_dict[index]]

    n_pad = max_len - len(input_ids)
    input_ids.extend([0] * n_pad)
    segment_ids.extend([0] * n_pad)

    if max_pred > n_pred:
      n_pad = max_pred - n_pred
      masked_tokens.extend([0] * n_pad)
      masked_pos.extend([0] * n_pad)

    if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
      batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])
      positive += 1
    elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
      batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])
      negative += 1

  return batch

In [40]:
def get_attn_pad_masked(seq_q, seq_k):
  batch_size, len_q = seq_q.size()
  batch_size, len_k = seq_k.size()
  pad_attn_masked = seq_k.data.eq(0).unsqueeze(1)
  return pad_attn_masked.expand(batch_size, len_q, len_k)

In [17]:
batch = make_batch()

In [18]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [19]:
input_ids[0]

tensor([ 1, 40, 45,  5, 52, 60,  3, 67, 19, 16, 62, 32,  2,  3, 44, 66, 34, 18,
        14, 68,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [20]:
segment_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [21]:
masked_tokens[0]

tensor([34,  4, 51,  0,  0,  0,  0])

In [22]:
masked_pos[0]

tensor([16, 13,  6,  0,  0,  0,  0])

In [23]:
isNext[0]

tensor(0)

In [24]:
get_attn_pad_masked(input_ids, input_ids)[0][0], input_ids[0]

(tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]),
 tensor([ 1, 40, 45,  5, 52, 60,  3, 67, 19, 16, 62, 32,  2,  3, 44, 66, 34, 18,
         14, 68,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  